In [ ]:
import pandas as pd
import numpy as np

fd = pd.read_csv("data.csv", parse_dates=[0])
print(fd.info())
print(fd.describe())

In [ ]:
#fd = fd.drop("Unnamed: 32", axis=1)

print(len(fd["UNIQUE_CARRIER"].unique()))
d_air = fd["DEST"].unique()
o_air = fd["ORIGIN"].unique()
print(len(np.intersect1d(d_air, o_air)))
print(f"INTERVAL: from {fd['FL_DATE'].min()} to {fd['FL_DATE'].max()}")

In [ ]:
fd = fd[fd["CANCELLED"]==0]
fd.head()


In [ ]:
fd.groupby(by="UNIQUE_CARRIER").size()

In [ ]:
fd.pivot_table(index="UNIQUE_CARRIER", values=["CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"], aggfunc=np.mean)

In [ ]:
fd=fd.assign(weekday=lambda x: x["FL_DATE"].dt.day_of_week)
#print(fd[["FL_DATE", "weekday"]])


fd=fd.assign(delaydelta=lambda x: x["ARR_DELAY"]-x["DEP_DELAY"])
#print(fd[["DEP_DELAY", "ARR_DELAY", "delaydelta"]])

arr_delay = fd["ARR_DELAY"]
dow = fd["weekday"]

import seaborn as sns
delay_dow = pd.concat([arr_delay, dow], axis=1)
print(delay_dow)
delay_dow_corr = delay_dow.corr()
print(delay_dow_corr)

sns.heatmap(delay_dow_corr)



In [ ]:
weekend_fd = fd[fd["weekday"]>=5][["UNIQUE_CARRIER", "ARR_DELAY"]]
normaldays_fd = fd[fd["weekday"]<5][["UNIQUE_CARRIER", "ARR_DELAY"]]

weekend_fd = weekend_fd.groupby(by="UNIQUE_CARRIER").mean()
normaldays_fd = normaldays_fd.groupby(by="UNIQUE_CARRIER").mean()

In [ ]:
new_df = weekend_fd.merge(normaldays_fd, on="UNIQUE_CARRIER").rename(columns={'ARR_DELAY_x': 'ARR_DELAY_weekend', 'ARR_DELAY_y': 'ARR_DELAY_week'})
print(new_df)
weekend_fd.name="weekend delay"
normaldays_fd.name="working days delay"
ax = pd.concat([normaldays_fd, weekend_fd], axis=1).plot.bar()
ax.grid(True)


In [ ]:
multi_df = fd.set_index(['UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'FL_DATE']).sort_index()
multi_df.head()


In [ ]:
multi_df.loc[(['AA', 'DL'], ['LAX']), ['DEP_TIME', 'DEP_DELAY']]

In [ ]:
fw_df = multi_df.loc[(slice(None), slice(None), 'LAX', slice('2017-01-01','2017-01-08')), :]
fw_df.groupby("FL_NUM")["ARR_DELAY"].mean()

In [ ]:
fd.pivot_table(index=["UNIQUE_CARRIER", "weekday"], values="FL_NUM", aggfunc="count")

In [ ]:
cfd = fd.pivot_table(index="UNIQUE_CARRIER", columns="weekday", values="FL_NUM", aggfunc="count")

In [ ]:
sns.heatmap(cfd.T.corr())

In [ ]:
cwm = fd.pivot_table(index="UNIQUE_CARRIER", columns="weekday", values="ARR_DELAY", aggfunc="mean")
cwm

In [ ]:
sns.heatmap(cwm.T.corr())

In [ ]:
df13 = fd[fd["UNIQUE_CARRIER"].isin(["HA", "DL", "AA", "AS"])]
#print(df13.head())
df13_pt = df13.pivot_table(index="UNIQUE_CARRIER", columns="weekday", values="delaydelta", aggfunc="mean")
df13_pt


In [ ]:
df13_pt.T.plot()